In [31]:
RANDOM_SEED = 17
SIZE = 3

In [32]:
from gx_utils import *
import logging
from random import seed, choice
from typing import Callable
import numpy as np

logging.basicConfig(format="%(message)s", level=logging.INFO)



In [33]:
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [52]:
def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    while state is not None and not goal_test(state):
        for a in possible_actions(state):
            new_state = result(state, a)
            cost = unit_cost(a)

            # Adding threshold for cost (to ignore too long solutions)
            # if state_cost[state] + cost > 30:
                # continue

            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    return list(reversed(path))

Graph search for the the n-puzzle

In [35]:
seed(RANDOM_SEED)

In [36]:
GOAL = State(np.array(list(range(1, SIZE**2)) + [0]).reshape((SIZE, SIZE)))
logging.info(f"Goal:\n{GOAL}")


def goal_test(state):
    return state == GOAL

Goal:
[[1 2 3]
 [4 5 6]
 [7 8 0]]


In [37]:
# (R, C) -> UP / RIGHT / DOWN / LEFT
MOVES = [np.array(_) for _ in [(-1, 0), (0, +1), (+1, 0), (0, -1)]]

In [38]:
def find_empty_space(board: np.ndarray):
    t = np.where(board == 0)
    return np.array([t[0][0], t[1][0]])


def is_valid(board: np.ndarray, action):
    return all(0 <= (find_empty_space(board) + action)[i] < board.shape[i] for i in [0, 1])


def possible_actions(state: State):
    return (m for m in MOVES if is_valid(state._data, m))

In [39]:
def result(state, action):
    board = state.copy_data()
    space = find_empty_space(board)
    pos = space + action
    board[space[0], space[1]] = board[pos[0], pos[1]]
    board[pos[0], pos[1]] = 0
    return State(board)

In [40]:
INITIAL_STATE = GOAL
# Shuffling 500 times starting from goal 
for r in range(500):
    INITIAL_STATE = result(INITIAL_STATE, choice(list(possible_actions(INITIAL_STATE))))
INITIAL_STATE

array([[5, 4, 7],
       [1, 0, 6],
       [2, 8, 3]])

Breadth-First

In [41]:
parent_state = dict()
state_cost = dict()

final = search(
    INITIAL_STATE,
    goal_test=goal_test,
    parent_state=parent_state,
    state_cost=state_cost,
    priority_function=lambda s: len(state_cost), # len(state_cost) will be the # of discovered states so far (FIFO)
    unit_cost=lambda a: 1, # All actions are worth 1
)

Found a solution in 27 steps; visited 174,246 states


Depth-First

In [53]:
parent_state = dict()
state_cost = dict()

final = search(
    INITIAL_STATE,
    goal_test=goal_test,
    parent_state=parent_state,
    state_cost=state_cost,
    priority_function=lambda s: -len(state_cost), # Lowest priority served, so in this case the deepest paths (LIFO)
    unit_cost=lambda a: 1,
)

Found a solution in 61,537 steps; visited 116,779 states


Greedy Best-First

In [43]:
parent_state = dict()
state_cost = dict()


def h(state):
    return np.sum((state._data != GOAL._data) & (state._data > 0))
    # Sum of the number of different cells between state and solution and +1 for the 0 cell (even if it's
    # placed in the same position) 
    
    # What is the second condition?

final = search(
    INITIAL_STATE,
    goal_test=goal_test,
    parent_state=parent_state,
    state_cost=state_cost,
    priority_function=lambda s: h(s),
    unit_cost=lambda a: 1,
)

Found a solution in 63 steps; visited 1,195 states


A*

In [46]:
parent_state = dict()
state_cost = dict()


def h(state):
    return np.sum((state._data != GOAL._data) & (state._data > 0))


final = search(
    INITIAL_STATE,
    goal_test=goal_test,
    parent_state=parent_state,
    state_cost=state_cost,
    priority_function=lambda s: state_cost[s] + h(s), # Considers not only the future, but the actual cost so far as well
    unit_cost=lambda a: 1,
)

Found a solution in 27 steps; visited 48,434 states


In [45]:
(INITIAL_STATE._data != GOAL._data) & (INITIAL_STATE._data > 0)

array([[ True,  True,  True],
       [ True, False, False],
       [ True, False,  True]])